In [4]:
import torch
import torch.nn as nn
from topTcbam_UNet import top_t_cbam_UNet


torch.Size([1, 1])


In [5]:
import matplotlib.pyplot as plt
import torch
import numpy as np
from metric import *
from torch.utils.data import DataLoader

In [13]:
import numpy as np
from PIL import Image
import torch as torch
from torch.utils.data import Dataset,DataLoader
import torchvision.transforms as transforms
import os
from einops import rearrange
from custom_transforms import *
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from monai.transforms import AsDiscrete


class tumor_Dataset_testing(Dataset):
    def __init__(self,path,patient, train=True):
        self.patientID = patient
        self.path = path
        self.train = train
        self.train_path_list = []
        self.train_list = []


        self.label_path_list = []
        self.label_list = []

        self.train_path = path + "/input"
        self.label_path = path + "/target"

        
        for file in os.listdir(self.train_path):
            if file.split("_")[0] == str(self.patientID):
                self.train_path_list.append(os.path.join(self.train_path,file))
        self.train_path_list.sort()
                
        for file in os.listdir(self.label_path):
            if file.split("_")[0] == str(self.patientID):
                self.label_path_list.append(os.path.join(self.label_path,file))
        self.label_path_list.sort()

    def __len__(self):
        return len(self.label_path_list)

    def preprocessing(self,train_path, label_path):

        input_slice = pydicom.read_file(train_path)
        input_img = input_slice.pixel_array
        input_img = apply_voi_lut(input_img, input_slice)
        epsilon = 1e-10
        min_val = np.min(input_img)
        max_val = np.max(input_img)
        input_img = (input_img - min_val) / (max_val - min_val+epsilon)
        input_img = Image.fromarray(input_img)

        target_slice = pydicom.read_file(label_path)
        target_img = target_slice.pixel_array
        epsilon = 1e-10
        min_val = np.min(target_img)
        max_val = np.max(target_img)
        target_img = (target_img - min_val) / (max_val - min_val+epsilon)

        target_img = Image.fromarray(target_img)

        return input_img, target_img

    def __getitem__(self,idx):
        if self.train:
            self.transform = transforms.Compose([transforms.ToTensor(),
                                                transforms.Resize((512,512)),
                                                customRandomRotate(degrees=180,SEED=idx),
                                                customRandomHorizontalFlip(p=0.5,SEED=idx),
                                                #customRandomResizedCrop(SEED=idx,size=(256,256))
                                                ])
        else:
            self.transform = transforms.Compose([transforms.ToTensor(),
                                                 transforms.Resize((512,512))
                                                 ])

        

        image,label = self.preprocessing(self.train_path_list[idx], self.label_path_list[idx])    

        input_image = self.transform((image))
        target_image = self.transform((label))

        threshold = AsDiscrete(threshold=0.5)
        target_image = threshold(target_image)


        return input_image, target_image

In [6]:
net_path = '/workspace/IITP/task_2D/dir_checkpoint_tumorResult/top_t_cbam_002_UNet_dataedited.pth'
net = top_t_cbam_UNet(1,1,percent_t=0.02).to(device="cuda:0")
if torch.cuda.device_count() > 1:
    net = nn.DataParallel(net,device_ids=[0,1,2,3]) 
net.load_state_dict(torch.load(net_path))

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

set_seed(7777777)

dataset = tumor_Dataset("/mount_folder/Tumors/test/undersampling")

In [14]:
import numpy as np
import torch
import pandas as pd


test_excel = pd.read_excel("/mount_folder/test_data.xlsx")
test_list = test_excel["Patient"].tolist()


def evaluate_segmentation(model, dataset, test_list, device='cuda:0'):
    # 초기 confusion matrix 요소 값
    TP_total = 0
    FP_total = 0
    FN_total = 0
    TN_total = 0
    
    model.eval()
    test_loader = DataLoader(dataset=dataset,batch_size=1,shuffle=False)

    for input_image, true_mask in test_loader:
        input_image = input_image.to(device)
        
        with torch.no_grad():
            predicted_mask = model(input_image)

        predicted_mask = predicted_mask.cpu().numpy()
        true_mask = true_mask.cpu().numpy()

        # 이진 분류를 위한 threshold 적용
        predicted_mask = (predicted_mask > 0.5).astype(np.uint8)
        
        # 현재 slice에 대한 confusion matrix 요소 계산
        TP = np.sum((predicted_mask == 1) & (true_mask == 1))
        FP = np.sum((predicted_mask == 1) & (true_mask == 0))
        FN = np.sum((predicted_mask == 0) & (true_mask == 1))
        TN = np.sum((predicted_mask == 0) & (true_mask == 0))
        
        # 현재 slice의 결과를 전체 결과에 합산
        TP_total += TP
        FP_total += FP
        FN_total += FN
        TN_total += TN
    
    # 모든 slices를 기반으로 한 Dice Score 계산
    dice = (2 * TP_total) / (2 * TP_total + FP_total + FN_total)
    
    return dice

# 사용 예시
# patient_slices = load_slices(patient_number="12345")  # patient_slices는 (input_image, true_mask) 튜플의 리스트
# dice_score = evaluate_segmentation(model, patient_slices)
# print(dice_score)


In [18]:
dice_dict = {}
for patient in test_list:
    dataset = tumor_Dataset_testing(path="/mount_folder/Tumors/test",patient=patient, train=False)
    model = net
    dice = evaluate_segmentation(model, dataset, test_list, device='cuda:0')
    print(f"{patient} : {dice}")
    dice_dict[f"{patient}"] = dice

1167753 : 0.8580580363202042
1199340 : 0.3412361244346992
1199440 : 0.6140790287886377
1199470 : 0.33448601068846273
1199870 : 0.02827521206409048
1199940 : 0.624931438544004
1219657 : 0.75208734746307
1229070 : 0.5512407652964577
1249647 : 0.8179940083376043
1284590 : 0.8392113063275914
1299050 : 0.8269854508386618
1299690 : 0.9107241876455584
1329450 : 0.8734185427885551
1329490 : 0.6980462589801998
1364554 : 0.0
1389134 : 0.38798977853492334
1399050 : 0.7254370844619552
1399230 : 0.8343834383438344
1399820 : 0.6897493174484984
1399990 : 0.7516377397885945
1430946 : 0.4187323240725337
1449557 : 0.8381968359182828
1499030 : 0.0
1499100 : 0.7543140028288543
1499560 : 0.8267552970502701
1587857 : 0.7281387858940702
2099330 : 0.8648466949986794
2296328 : 0.8717900087243485
2319117 : 0.0
2399130 : 0.3109699486928903
2559967 : 0.6833134159381093
2697891 : 0.9164044943820224
2709627 : 0.8801586518184836
2899910 : 0.911361804995971
3134327 : 0.8833291213882571
3341384 : 0.8558036615359313
33

In [16]:
dice_dict

{}